# Geo Plotting

Inspired by [https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas](https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas)

In [ ]:
!pip install geopandas plotly seaborn

In [7]:
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# opt in to ignore warnings
import warnings
warnings.filterwarnings("ignore")

# get map data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
# Map: Admin 0 – Countries
# https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip

import os
cwd = os.getcwd()
world = gpd.read_file(f"zip://{cwd}/ne_110m_admin_0_countries.zip")

## Map plotting

Geopandas uses classes from [shapely](https://shapely.readthedocs.io/en/stable/index.html),
like:
* [Points](https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html)
* [Polygon](https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon)
* [MultiPolygon](https://shapely.readthedocs.io/en/stable/reference/shapely.MultiPolygon.html)

### Plotting a single connection

#### Get shapely datas

In [8]:
sa = world[world["NAME"].str.contains("South Africa")]
uk = world[world["NAME"].str.contains("United Kingdom")]
sa.iloc[0]["NAME"]

'South Africa'

In [ ]:
# print plot styles
plt.style.available

In [20]:
cmap = plt.colormaps["GnBu"]
from matplotlib.colors import Normalize

def plot_country_name(ax, country):
    name = country.iloc[0]["NAME"]
    country_x = country.centroid.x + 5
    country_y = country.centroid.y - 5
    ax.text(country_x, country_y, name, fontsize=8, color="black", alpha=0.8, horizontalalignment='center', verticalalignment='center')

def plot_country(ax, country, color_scalar: float):
    country["geometry"].plot(ax=ax, color=cmap(color_scalar))

def plot_connection(ax, country_a, country_b, linewidth=1, alpha=.5):
    a = country_a["geometry"].centroid
    b = country_b["geometry"].centroid
    ax.plot([a.x , b.x], [a.y, b.y], color="black", linewidth=linewidth, alpha=alpha)
    ax.scatter(a.x, a.y, color="black", s=10)
    ax.scatter(b.x, b.y, color="black", s=10)

with plt.style.context(("bmh")):
    ax = world.plot(figsize=(18,10), edgecolor="grey", color=cmap(.0));
    fig = ax.get_figure()
    fig.set_dpi(300)
    fig.colorbar(plt.cm.ScalarMappable(norm=Normalize(0, 1), cmap=cmap), ax=ax, label="a colorbar")
    plot_country(ax, sa, .4)
    plot_country(ax, uk, .8)
    plot_country_name(ax, uk)
    plot_country_name(ax, sa)
    plot_connection(ax,uk,sa)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title("Connection between some points on a map")
    plt.savefig("file.png", dpi=100)

### Create some random connections

In [11]:
import random

def random_country_connections(world: gpd.GeoDataFrame):
    c_connections = []
    for i in range(50):
        index_a = random.randint(0,len(world)-1)
        country_a = world.iloc[index_a]["NAME"]
        connections = [{"Country": world.iloc[random.randint(0,len(world)-1)]["NAME"],
                        "Count": random.randint(1,5)}
                        for _ in range(random.randint(1,5))]
        country_dict = {"Country": country_a, "Connections": connections}
        c_connections.append(country_dict)
    return c_connections

### Plot heatmap based on the connection count

In [18]:
def find_country_by_name(country_name):
    country = world[world["NAME"].str.contains(country_name)]
    if len(country) > 1:
        print("Country name is ambigous")
        raise Exception
    return country
import numpy

connections = random_country_connections(world)
max_connections = max([len(i["Connections"]) for i in connections])
max_multi_connections = max([j["Count"] for i in connections for j in i["Connections"]])

with plt.style.context(("bmh")):
    ax = world.plot(figsize=(18,10), edgecolor="grey", color=cmap(.0));
    fig = ax.get_figure()
    fig.set_dpi(200)
    fig.colorbar(plt.cm.ScalarMappable(norm=Normalize(0, 1), cmap=cmap), ax=ax, label="a colorbar")
    for c in connections[0:10]:
        try:
            src = find_country_by_name(c["Country"])
        except Exception:
            continue # ignore error and continue
        src_color_scalar = len(c["Connections"]) / max_connections
        plot_country(ax, src, src_color_scalar)
        if len(c["Connections"]) / max_connections > 0.5:
            plot_country_name(ax, src)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title("Geo heatmap")
    #plt.savefig("heatmap.png", dpi=100)

In [19]:
connections = random_country_connections(world)
max_connections = max([len(i["Connections"]) for i in connections])
max_multi_connections = max([j["Count"] for i in connections for j in i["Connections"]])

with plt.style.context(("bmh")):
    ax = world.plot(figsize=(18,10), edgecolor="grey", color=cmap(.0));
    fig = ax.get_figure()
    c = connections[0]
    print(c)
    src = find_country_by_name(c["Country"])
    src_color_scalar = len(c["Connections"]) / max_connections
    plot_country(ax, src, src_color_scalar)
    plot_country_name(ax, src)
    for connection in c["Connections"]:
        try:
            dest = find_country_by_name(connection["Country"])
        except Exception:
            print()
            continue # ignore error and continue
        plot_country_name(ax, dest)
        alpha = connection["Count"] / max_multi_connections
        plot_connection(ax, src, dest, alpha=alpha)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title("Connection between some points on a map")
    #plt.savefig("connections.png", dpi=100)

{'Country': 'Portugal', 'Connections': [{'Country': 'Palestine', 'Count': 2}]}
